# Traitement des données en tables

**Plan du document**
- **Pré-traitement** des données
- **Projection**: sélection de «colonnes» - descripteurs
- **Sélection**: sélection de «lignes» - objets
    - fonction qui prend en argument ... une autre fonction!
- **Trier** les données

Dans cette section, nous supposons disposer d'une table de n-uplets nommés (à l'aide de dictionnaires) et donc de la forme:

`[{descr1: val1, descr2: val2,...},{descr1: val'1, ...}, ...]`

qui représente le tableau (qui peut avoir plus de trois colonnes!)

| descr1   | descr2           | ...  |
| :-------------:|:-------------:|:-----:|
| val1      | val2 | ... |
| val'1      | ...      |   ... |
| ... | ...   |    ... |

**Objectif**: apprendre à réaliser certaines opérations courantes sur ce genre de données:
- pré-traitements
- sélectionner des enregistrements (lignes) - **sélection**,
- sélectionner des descripteurs (colonnes) - **projection**,
- **trier** les enregistrements (lignes) sur la base d'un ou de plusieurs descripteurs.

Voici la table que nous utiliserons pour tester ces opérations.

In [ ]:
table_test = [
    {'n_client': '1212', 'nom': 'Lacasse'  , 'prenom': 'Aubrey'   , 'ville': 'Annecy'  , 'position': '45.900000,6.116667'},
    {'n_client': '1343', 'nom': 'Primeau'  , 'prenom': 'Angelette', 'ville': 'Tours'   , 'position': '47.383333,0.683333'},
    {'n_client': '2454', 'nom': 'Gabriaux' , 'prenom': 'Julie'    , 'ville': 'Bordeaux', 'position': '44.833333,-0.566667'},
    {'n_client': '895' , 'nom': 'Gaulin'   , 'prenom': 'Dorene'   , 'ville': 'Lyon'    , 'position': '45.750000,4.850000'},
    {'n_client': '2324', 'nom': 'Jobin'    , 'prenom': 'Aubrey'   , 'ville': 'Bourges' , 'position': '47.083333,2.400000'},
    {'n_client': '34'  , 'nom': 'Boncoeur' , 'prenom': 'Kari'     , 'ville': 'Nantes'  , 'position': '47.216667,-1.550000'},
    {'n_client': '1221', 'nom': 'Parizeau' , 'prenom': 'Olympia'  , 'ville': 'Metz'    , 'position': '49.133333,6.166667'},
    {'n_client': '1114', 'nom': 'Paiement' , 'prenom': 'Inès'     , 'ville': 'Bordeaux', 'position': '44.833333,-0.566667'},
    {'n_client': '3435', 'nom': 'Chrétien' , 'prenom': 'Adèle'    , 'ville': 'Moulin'  , 'position': '46.566667,3.333333'},
    {'n_client': '5565', 'nom': 'Neufville', 'prenom': 'Ila'      , 'ville': 'Toulouse', 'position': '43.600000,1.433333'},
    {'n_client': '2221', 'nom': 'Larivière', 'prenom': 'Alice'    , 'ville': 'Tours'   , 'position': '47.383333,0.683333'},
]

Quels sont les descripteurs de cette table? **réponse**: ______

In [ ]:
# combien comporte-t-elle d'«objets»? réponse: ____


Quel est le type commun de toutes les valeurs? **réponse**: ______

#### Parcours d'un dictionnaire

Lorsqu'on utilise la syntaxe `for <var> in <dictionnaire>`, la variable de boucle contient une nouvelle **clé** du dictionnaire à chaque itération.

In [ ]:
test = {"un": 1, "deux": 2, "3": "trois"}
for var in test:
    print(var)

À partir de la clé, on peut facilement récupérer la valeur associée dans la paire clé-valeur avec la syntaxe `dico[cle]`:

In [ ]:
test = {"un": 1, "deux": 2, "3": "trois"}
for var in test:
    print(test[var])

Mais il est plus pratique de récupérer directement **la clé et la valeur** dans la variable de boucle. On peut faire cela en utilisant la méthode `dico.items()` dans la boucle:

In [ ]:
test = {"un": 1, "deux": 2, "3": "trois"}
for var in test.items():
    print(var)

Comme à chaque itération, la variable de boucle reçoit un *tuple de taille 2*, on peut récupérer directement chaque composante comme suit:

In [ ]:
test = {"un": 1, "deux": 2, "3": "trois"}
for cle, val in test.items():
    print(f"{cle} => {val}")

**Retenir**

> si `d` est un dictionnaire, `for cle, val in d.items()` récupère une nouvelle paire clé-valeur à chaque itération de la boucle.

## Pré-traitement ou préparation des données

On observe que certains **descripteurs** pourrait avoir un **type** plus précis que `str`, par exemple le descripteur `n_client` gagnerait a être de type `int`.

Améliorons cela à l'aide de la syntaxe en compréhension:

In [ ]:
table_test_2 = [ # liste en compréhension: produit une liste de ...
    { # ... dictionnaires ...
        c: int(v) if c == 'n_client' else v # conversion de la valeur si le descripteur est 'n_client'
        for c, v in l.items() 
    }
    for l in table_test # sur la base de chaque ligne de la table d'origine
]
# observe bien la valeur du descripteur 'n_client'
table_test_2[:2]

C'est encore un peu difficile à lire probablement? l'opérateur ternaire `e1 if cond else e2` n'est pas encore parfaitement clair? ni l'écriture en compréhension?

Alors voici l'équivalent dans une fonction avec une boucle imbriquée.

In [ ]:
def conversion1(table):
    tc = []  # table à construire
    # pour chaque enregistrement
    for l in table:
        enr = {} # nouvel enregistrement
        # pour chaque paire clé-valeur de l'enregistrement courant
        for c, v in l.items():
            # doit-on convertir en int?
            if c == 'n_client':
                enr[c] = int(v)
            else:
                enr[c] = v
        # ajouter le nouvel enregistrement à la table en consruction
        tc.append(enr)
    return tc

conversion1(table_test)

Mais il y a un autre descripteur qui pose problème: `position`

son type est `str` au format `'<float>,<float>'`

nous voudrions que son type soit «`tuple` de `float`» c'est-à-dire passer de (par ex.) `'45.900000,6.116667'` à `(45.900000, 6.116667)`.

### À toi de jouer

En t'inspirant de la conversion résolue précédente, transforme le descripteur `position` en un tuple de 2 floats.

In [ ]:
# avec une fonction
def conversion2(table):
    pass

In [ ]:
# avec la syntaxe en compréhension (tu peux utiliser plusieurs étapes)


_________

## Projection ou sélection de colonnes

On repart de la table ci-dessous:

In [ ]:
table_test = [
    {'n_client': 1212, 'nom': 'Lacasse'  , 'prenom': 'Aubrey'   , 'ville': 'Annecy'  , 'position': (45.900000,6.116667)},
    {'n_client': 1343, 'nom': 'Primeau'  , 'prenom': 'Angelette', 'ville': 'Tours'   , 'position': (47.383333,0.683333)},
    {'n_client': 2454, 'nom': 'Gabriaux' , 'prenom': 'Julie'    , 'ville': 'Bordeaux', 'position': (44.833333,-0.566667)},
    {'n_client': 895 , 'nom': 'Gaulin'   , 'prenom': 'Dorene'   , 'ville': 'Lyon'    , 'position': (45.750000,4.850000)},
    {'n_client': 2324, 'nom': 'Jobin'    , 'prenom': 'Aubrey'   , 'ville': 'Bourges' , 'position': (47.083333,2.400000)},
    {'n_client': 34  , 'nom': 'Boncoeur' , 'prenom': 'Kari'     , 'ville': 'Nantes'  , 'position': (47.216667,-1.550000)},
    {'n_client': 1221, 'nom': 'Parizeau' , 'prenom': 'Olympia'  , 'ville': 'Metz'    , 'position': (49.133333,6.166667)},
    {'n_client': 1114, 'nom': 'Paiement' , 'prenom': 'Inès'     , 'ville': 'Bordeaux', 'position': (44.833333,-0.566667)},
    {'n_client': 3435, 'nom': 'Chrétien' , 'prenom': 'Adèle'    , 'ville': 'Moulin'  , 'position': (46.566667,3.333333)},
    {'n_client': 5565, 'nom': 'Neufville', 'prenom': 'Ila'      , 'ville': 'Toulouse', 'position': (43.600000,1.433333)},
    {'n_client': 2221, 'nom': 'Larivière', 'prenom': 'Alice'    , 'ville': 'Tours'   , 'position': (47.383333,0.683333)},
]

Cette opération consiste à «oublier» un ou plusieurs descripteurs (et les valeurs associées) pour chaque enregistrement. Par exemple, pour se concentrer sur ceux qui nous intéresse dans notre projet.

Le problème est le suivant: étant donnée une liste de descripteurs à oublier, produire la table de donnée correspondante.

ex: si `a_oublier = ['n_client', 'prenom', 'position']` alors l'enregistrement:
- `{'n_client': 1212, 'nom': 'Lacasse', 'prenom': 'Aubrey', 'ville': 'Annecy', 'position': (45.900000,6.116667)}` ...
- ... devient `{'nom': 'Lacasse', 'ville': 'Annecy'}` ...
- et ainsi de suite pour chaque enregistrement.

In [ ]:
def projection_par_oubli(tableau, a_oublier):
    tsel = [] # pour notre nouveau tableau
    # pour chaque enregistrement
    for l in tableau:
        enr = {} # pour notre nouvel enregistrement
        # pour chaque paire clé-valeur de l'enregistrement courant
        for c, v in l.items():
            if not c in a_oublier:
                # on conserve cette paire
                enr[c] = v
        # ajoutons notre nouvel enregistrement
        tsel.append(enr)
    return tsel

projection_par_oubli(table_test, ['n_client', 'prenom', 'position'])

### À ton tour

1. Peux-tu réaliser la même chose avec la notation en compréhension en une ou plusieurs étapes?

2. Écris une fonction `projection(tableau, a_conserver)` qui prend en argument le tableau de données et la liste des descripteurs à conserver; elle renvoie le tableau «projeté».

_____

## Sélection d'enregistrement

On souhaite à présent transformer le tableau en ne conservant que les *enregistrements* qui respectent *un certain critère*; autrement dit on veut sélectionner certaine lignes.

Par exemple, on pourrait vouloir sélectionner les clients qui habitent à tours.

In [ ]:
def selection_exemple(tableau):
    tsel = []
    for l in tableau:
        if l['ville'] == 'Tours':
            tsel.append(l)
    return tsel

selection_exemple(table_test)

### À ton tour!

1. écris une fonction `selection2` qui renvoie le tableau en ne conservant que les objets dont le numéro de client est dans l'intervalle `[1000;3000]`.

2. écris une fonction `selection3` qui sélectionne les objets dont la longitude est positive - 'pos=(lat., long.)' - et dont le prénom débute par un 'A'.

____

### Une fonction qui prend en argument une autre fonction!

Il est simple d'adapter le code précédent pour sélectionner selon un autre critère, mais on peut faire bien mieux en suivant ces étapes:
1. Définir une fonction qui, étant donné un enregistrement, renvoie `True` si l'enregistrement respecte le critère, `False` autrement - une telle fonction est appelée un **filtre**.
2. Passer en argument la fonction filtre à la fonction de selection.

Commençons par l'**étape 2** :-o

In [ ]:
def selection(tableau, filtre_fn):
    tsel = []
    for l in tableau:
        if filtre_fn(l): # rappel filtre est une fonction qui renvoie `True` ou `False`
            tsel.append(l)
    return tsel

Pour l'**étape 1**, une «micro fonction» suffit bien souvent:

In [ ]:
a_tours = lambda enregistrement: enregistrement['ville'] == 'Tours'

Finalement, on combine les deux:

In [ ]:
selection(table_test, a_tours)

En fait, on peut se passer de l'étape 2 et c'est justement la raison d'être des «micro fonctions» appelée parfois *fonction anonyme*, il suffit d'écrire:

In [ ]:
# sur plusieurs lignes pour plus de clarté; remettre sur une ligne. 
selection(
    table_test,
    lambda o: o['ville'] == 'Tours'
)

Si le filtre est plus compliqué, rien n'empêche d'utiliser une fonction «normale»

In [ ]:
def filtre_tordu(objet):
    condition1 = objet['ville'] == 'Tours'
    condition2 = objet['nom'][0] in ['P', 'B']
    return condition1 or condition2

# en fait, on pourrait encore utiliser une «micro-fonction» dans ce cas.
selection(table_test, filtre_tordu)

#### À ton tour!

Écris les micros fonctions ou des fonctions ordinaire qui permettent de résoudre le dernier «À ton tour» 

_____

Rencontrer pour la première fois «une fonction qui prend en argument une autre fonction» - parfois appelée **fonction d'ordre supérieur** - est souvent déroutant.

Pour «passer le cap», voici une activité complémentaire.

#### Activité: ma première fonction d'ordre supérieur

Écris une fonction `appliquer(liste, fn)` qui prend en argument:
- une liste d'éléments de type 'a': ce type est arbitraire, l'important c'est que tous les éléments de la liste aient le même type,
- une fonction `fn` qui prend en argument un élément de type 'a' et renvoie un élément de type 'b'.

Finalement, la fonction `appliquer` renvoie une liste d'éléments de type 'b' (tous)

**En résumé**: `appliquer` reçois `liste: "[a]"` et `fn: "a -> b"` et elle produit `"[b]"`...

Aide-toi des assertions qui suivent pour résoudre le problème.

In [ ]:
def appliquer(liste, fn):
    # à toi de jouer!

In [ ]:
l = [1,2,3]
f = lambda x: x**2 # f: int -> int
assert appliquer(l, f) == [1,4,9]
l = ["un", "deux", "trois"]
f = lambda ch: len(ch) # f: str -> int
assert appliquer(l, f) == [2,4,5]
f = lambda ch: ch.upper() # f: str -> str
assert appliquer(l, f) == ["UN", "DEUX", "TROIS"]

_____

## Trier le tableau selon un ou plusieurs descripteurs

In [ ]:
table_test = [
    {'n_client': 1212, 'nom': 'Lacasse'  , 'prenom': 'Aubrey'   , 'ville': 'Annecy'  , 'position': (45.900000,6.116667)},
    {'n_client': 1343, 'nom': 'Primeau'  , 'prenom': 'Angelette', 'ville': 'Tours'   , 'position': (47.383333,0.683333)},
    {'n_client': 2454, 'nom': 'Gabriaux' , 'prenom': 'Julie'    , 'ville': 'Bordeaux', 'position': (44.833333,-0.566667)},
    {'n_client': 895 , 'nom': 'Gaulin'   , 'prenom': 'Dorene'   , 'ville': 'Lyon'    , 'position': (45.750000,4.850000)},
    {'n_client': 2324, 'nom': 'Jobin'    , 'prenom': 'Aubrey'   , 'ville': 'Bourges' , 'position': (47.083333,2.400000)},
    {'n_client': 34  , 'nom': 'Boncoeur' , 'prenom': 'Kari'     , 'ville': 'Nantes'  , 'position': (47.216667,-1.550000)},
    {'n_client': 1221, 'nom': 'Parizeau' , 'prenom': 'Olympia'  , 'ville': 'Metz'    , 'position': (49.133333,6.166667)},
    {'n_client': 1114, 'nom': 'Paiement' , 'prenom': 'Inès'     , 'ville': 'Bordeaux', 'position': (44.833333,-0.566667)},
    {'n_client': 3435, 'nom': 'Gabriaux' , 'prenom': 'Adèle'    , 'ville': 'Moulin'  , 'position': (46.566667,3.333333)},
    {'n_client': 5565, 'nom': 'Neufville', 'prenom': 'Ila'      , 'ville': 'Toulouse', 'position': (43.600000,1.433333)},
    {'n_client': 2221, 'nom': 'Larivière', 'prenom': 'Alice'    , 'ville': 'Tours'   , 'position': (47.383333,0.683333)},
]

Supposer que nous souhaitions **ordonner** les objets selon leur descripteur `n_client`: du plus petit numéro au plus grand.

Pour faire cela, nous utiliserons la fonction prédéfinie de python `sorted(liste, key=choix_descripteur_fn)` qui renvoie la liste `liste` triée suivant la *fonction* `choix_descripteur_fn` (la liste initiale n'est pas modifiée).

Pour notre exemple, la fonction est `lambda o: o['n_client']`: elle prend un objet (une ligne) et renvoie la valeur de son descripteur `'n_client'`.

In [ ]:
triee1 = sorted(table_test, key=lambda o: o['n_client']) # argument nommé key
triee1

Si nous souhaitons trier les objets (clients) suivant leur nom, **puis** leur prenom, nous renvoyons un tuple dans cet ordre.

In [ ]:
sorted(
    table_test,
    key=lambda o: (o['nom'], o['prenom']) # les parenthèses sont en fait inutiles
)

Voyez-vous la différence si nous trions seulement sur le nom? (observez bien).

### À faire vous-même

1. Trier la table selon la première lettre du prénom puis selon la longitude (Est vers Ouest).

   *Rappel*: position=(lat,long)

2. Sachant que `sorted` prend un troisième argument optionnel nommé `reverse` et qui vaut `False` par défaut, trier la table selon le numéro de client dans l'ordre descendant (plus grand en premier).